In [ ]:
import os, sys
current = os.path.dirname(os.path.realpath("*.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent)
%cd ..

In [ ]:
# !python mnist.py --num-epochs 10 

In [ ]:
from mnist import *

In [ ]:
device = torch.device(
    'cuda' if torch.cuda.is_available() else
    'mps' if torch.backends.mps.is_available() else
    'cpu'
)

# data loaders
train_dataset = datasets.MNIST(
    root='data', download=True, train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(
    root='data', download=True, train=False, transform=transforms.ToTensor())

# training settings
lot_size = 600

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=lot_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=lot_size, shuffle=False)

model = LinearNet(in_features=784, hidden=100)

# loss function
criterion = nn.CrossEntropyLoss()

lr = 0.05
num_epochs = 20

# differentially private optimizer
optimizer = optim.DPSGD(model.named_parameters(), lot_size, lr=lr, noise_scale=2,
                        max_grad_norm=4)

num_batches = len(train_loader)

logger = {'loss': [], 'total_loss': [], 'accuracy': [], 'total_accuracy': [], 'total_val_accuracy': []}

In [ ]:
train_dp_model(model, criterion, optimizer, num_epochs, num_batches, train_loader, test_loader, device=device,
               logger=logger)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].plot(logger['accuracy'])
ax[0].set_title('accuracy')

ax[1].plot(logger['total_accuracy'], label='train accuracy')
ax[1].set_title('total accuracy')
ax[1].plot(logger['total_val_accuracy'], label='val accuracy')
ax[1].legend()
plt.show()